In [1]:
!pip install plotly
!pip install yfinance
!pip install hvplot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 6.4 MB/s eta 0:00:00


In [2]:
import yfinance as yf

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.style as style
import seaborn as sns
# import plotly.graph_objects as go
import hvplot.pandas
import holoviews as hv

import datetime
from datetime import date, timedelta

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:

# Data for AstraZeneca (AZN.L)
azn = yf.Ticker("AZN.L")

In [4]:
# Download data for 2010-2019
azn_historical =  yf.download("AZN.L", start=datetime.datetime(2010, 1, 1),
                                     end=datetime.datetime(2023, 1, 1))

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
azn_historical


Price,Close,High,Low,Open,Volume
Ticker,AZN.L,AZN.L,AZN.L,AZN.L,AZN.L
Date,,,,,
2010-01-04,2923.263672,2928.233518,2894.438563,2894.438563,2091679
2010-01-05,2889.965576,2931.712284,2883.504776,2918.293700,2869739
2010-01-06,2857.661133,2891.456082,2833.308890,2891.456082,3136541
2010-01-07,2892.947510,2907.857049,2861.637478,2861.637478,3748200
2010-01-08,2890.959961,2909.845379,2872.074543,2903.384578,2473846
...,...,...,...,...,...
2022-12-22,11299.614258,11361.579240,11195.672998,11229.653795,2227678
2022-12-23,11231.652344,11321.601508,11199.670419,11283.622972,440196


In [6]:
# Create DataFrame dropping all but Date and Close columns
signals_df = azn_historical.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])

KeyError: 'Adj Close'

### SMA Crossover Strategy

The moving average crossover trading strategy will be to take two moving averages - 50-day (fast) and 100-day (slow) - and to go long (buy) when the fast MA goes above the slow MA and to go short (sell) when the fast MA goes below the slow MA.

In [ ]:
# 50 and 100 day SMA

short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['Close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Print the DataFrame
signals_df.tail(10)

In [ ]:
# Colab rendering requires reloading extension in each cell

def _render(self, **kw):
  hv.extension('bokeh')
  return hv.Store.render(self)
hv.core.Dimensioned._repr_mimebundle_ = _render

In [ ]:
# Visualise exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price',
    marker = 'v',
    width=1000,
    height=400
)

# Visualise entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price',
    marker = '^',
    width=1000,
    height=400
)

# Visualise close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price',
    width=1000,
    height=400
)

# Visualise moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

In [ ]:
# set an initial investment stake of capital and set the number of shares - here 500 shares of AZN.L
# Set initial capital
initial_capital = float(1000000)

# Set the share size
share_size = 500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(10)

In [ ]:
# Visualise exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualise entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualise total portfolio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

## Backtesting

In [ ]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialise the DataFrame with index set to evaluation metrics and column as 'Backtest'
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

In [ ]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualised return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

In [ ]:
import pandas as pd

# Initialise trade evaluation DataFrame with an empty DataFrame
trade_evaluation_df = pd.DataFrame(columns=[
    'Stock',
    'Entry Date',
    'Exit Date',
    'Shares',
    'Entry Share Price',
    'Exit Share Price',
    'Entry Portfolio Holding',
    'Exit Portfolio Holding',
    'Profit/Loss']
)

trade_evaluation_df

In [ ]:
import pandas as pd

# Initialize an empty list to store trade records
trade_records = []

# Loop through signal DataFrame
# If 'Entry/Exit' is 1, set entry trade metrics
# Else if 'Entry/Exit' is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade records list
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = entry_portfolio_holding - exit_portfolio_holding

        # Append trade record to the list
        trade_records.append({
            'Stock': 'AZN.L',
            'Entry Date': entry_date,
            'Exit Date': exit_date,
            'Shares': share_size,
            'Entry Share Price': entry_share_price,
            'Exit Share Price': exit_share_price,
            'Entry Portfolio Holding': entry_portfolio_holding,
            'Exit Portfolio Holding': exit_portfolio_holding,
            'Profit/Loss': profit_loss
        })

# Convert the list of trade records to a DataFrame
trade_evaluation_df = pd.DataFrame(trade_records)

# Print the DataFrame
print(trade_evaluation_df)



In [ ]:
price_df = signals_df[['Close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(title='AZN.L',xaxis=None)

In [ ]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table


# Trading Dashboard using multiple stock tickers

In [ ]:

aqr3_stocks3= yf.download(" AZN.L GSK.L SBIN.NS JIOFIN.NS AXS PANW", start=datetime.datetime(2010, 1, 1),
                          end=datetime.datetime(2015,1,1), group_by='tickers')
aqr3_stocks3

In [ ]:
panw_historical =  aqr3_stocks3['PANW']
axs_historical =   aqr3_stocks3['AXS']
jio_historical =   aqr3_stocks3['JIOFIN.NS']
azn_historical =   aqr3_stocks3['AZN.L']
gsk_historical =   aqr3_stocks3['GSK.L']
sbin_historical =  aqr3_stocks3['SBIN.NS']

In [ ]:
# DataFrame of the closing price of selected stocks

aqr3_stocks3_close = pd.DataFrame({"PANW": panw_historical.Close,
                                "AXS": axs_historical.Close,
                                "JIOFIN.NS": jio_historical.Close,
                                "AZN.L": azn_historical.Close,
                                "GSK.L": gsk_historical.Close,
                                "SBIN.NS": sbin_historical.Close
                                })
aqr3_stocks3_close.head()